In [1]:
import pandas as pd
import numpy as np

#names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
Wetter_Eki=pd.read_csv("Wetter/2024_Stunde_117_Ehrenkirchen.csv",sep=';',decimal=",",header=0,na_values='-').fillna(0)

In [2]:
#Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
#Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
#Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
Wetter_Eki['Datum']=pd.to_datetime(Wetter_Eki['Tag']+' '+Wetter_Eki['Stunde']+' +0100',format='%d.%m.%Y %H:%M %z',utc=True)
#Wetter_Eki['Datum']=Wetter_Eki['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Wetter_Eki=Wetter_Eki.set_index('Datum')

In [104]:
names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)
DayAheadPrices['Datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('Datum')
preis_menge=DayAheadPrices
preis_menge[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]=Wetter_Eki[['AVG_TA200','AVG_WV200','MAX_WV200max','SUM_GS200']]

In [105]:
preis_menge['AVG_WV200'].mean(),preis_menge['MAX_WV200max'].mean()

(1.5477345173041894, 3.3853483606557377)

In [106]:
Windkraft_je_ms=pd.read_csv('Windanlagen_kw_pro_ms.csv',sep=';')
Windkraft_je_ms['max W/m²']=Windkraft_je_ms.apply(lambda row: pow(min(row['m/s'],15),3)*0.5*1.2, axis= 1)
Windkraft_je_ms['m/s cp']=Windkraft_je_ms['m/s']
Windkraft_je_ms.set_index('m/s',drop=True,inplace=True)
Windkraft_je_ms['rel_V172-7.2']=Windkraft_je_ms['V172-7.2']/Windkraft_je_ms['V172-7.2'].max()
Windkraft_je_ms.head()


,V162-5.6,V162-6.2,V172-7.2,V172-7.2SO,EK100,1998_600kw,E-126,Kitepower_Hawk,Kitepower_Falcon,SkySails_PN-14,max W/m²,m/s cp,rel_V172-7.2
m/s,,,,,,,,,,,,,
0.0,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.000,0.0,0.0
0.5,0,0,0,0,0.0,0,0,-7.0,-4.0,0.0,0.075,0.5,0.0
1.0,0,0,0,0,0.0,0,0,-6.5,-3.0,0.0,0.600,1.0,0.0
1.5,0,0,0,0,0.0,0,0,-6.0,-2.0,0.0,2.025,1.5,0.0
2.0,0,0,0,0,0.0,0,0,-5.5,-0.5,0.0,4.800,2.0,0.0


In [107]:
faktor=5.7
fak='{}'.format(faktor)
#faktor=6.5
max_wind=500
preis_menge['Wind '+fak+' m/s']=round(faktor*preis_menge['MAX_WV200max']/preis_menge['MAX_WV200max'].mean()*2)/2
preis_menge['Wind '+fak+' m/s']=preis_menge.apply(lambda row: min(row['Wind '+fak+' m/s'],max_wind),axis= 1)

preis_menge['max W/m²']=preis_menge.apply(lambda row: pow(min(row['Wind '+fak+' m/s'],15),3)*0.5*1.2, axis= 1)
preis_menge['pow H'+fak+' V172']=preis_menge['Wind '+fak+' m/s'].map(Windkraft_je_ms['rel_V172-7.2'])
preis_menge['Wert H'+fak+' V172']=preis_menge['pow H'+fak+' V172']*preis_menge['DE-LU Day Ahead EUR/MWh']/1000
preis_menge['z Anzahl']=1

In [108]:
pm_pv=preis_menge['SUM_GS200']
pm_preis=preis_menge['DE-LU Day Ahead EUR/MWh']
pm_wind=preis_menge['Wind '+fak+' m/s']

preis_menge['kat']=(np.where(pm_preis<0,'0: <0 ct/kWh',np.where(pm_pv>300,'1: PV > 300 W/m2',np.where(pm_preis<35,'2: <3,5 ct/kWh',np.where(pm_wind<faktor,'3: Wind < '+fak,'9: Rest')))))

In [109]:
preis_menge['Wind '+fak+' m/s'].mean(),preis_menge['max W/m²'].mean(),preis_menge['pow H'+fak+' V172'].sum(),preis_menge['Wert H'+fak+' V172'].sum()

(5.677823315118397, 252.2981045081967, 2103.489722222222, 132.64719511805555)

In [110]:
preis_menge['Monat']=preis_menge.index.month
preis_menge['Jahr']=preis_menge.index.year
Strom=pd.pivot_table(preis_menge,index='Monat', columns=['kat'],values=['Wert H'+fak+' V172','pow H'+fak+' V172','z Anzahl'], aggfunc='sum',margins=True,margins_name='2024')

In [111]:
Strom

Wert H5.7 V172                                                \
kat     0: <0 ct/kWh 1: PV > 300 W/m2 2: <3,5 ct/kWh 3: Wind < 5.7   
Monat                                                                
1          -0.016300         0.203871       0.319252      1.038125   
2          -0.000280         1.128826       0.505976      0.698445   
3          -0.014436         1.769830       0.336139      0.982607   
4          -0.598645         2.643492       0.275002      1.133174   
5          -0.229443         1.979697       0.236623      1.070332   
6          -0.356173         3.065271       0.052289      1.435002   
7          -0.200685         1.017329       0.062966      0.916749   
8          -0.105274         1.049623       0.021504      1.000879   
9          -0.018706         0.816191       0.233363      1.053080   
10         -0.005689         0.313994       0.166366      1.249957   
11         -0.001415         0.393710       0.159304      1.346830   
12         -0.004847         0.128324       0.846565      1.329706   
2024       -1.551892        14.510157       3.215348     13.254887   

                              pow H5.7 V172                                  \
kat       9: Rest        2024  0: <0 ct/kWh 1: PV > 300 W/m2 2: <3,5 ct/kWh   
Monat                                                                         
1       17.427243   18.972191     15.615972         3.269167      30.763056   
2       16.402228   18.735194      4.000000        26.991806      38.881944   
3        9.973575   13.047715      3.983333        37.276389      26.526111   
4       11.453310   14.906334     28.628472        53.761944      32.135139   
5        6.979724   10.036933     21.028194        30.973750      15.007778   
6        2.768753    6.965143     18.617778        45.916806       8.891806   
7        1.458721    3.255081     15.517500        19.559722       6.543611   
8        1.332993    3.299724     11.675694        16.815972       1.772083   
9        5.351538    7.435465     11.407361        17.924861      17.322361   
10       3.151161    4.875789      7.542361         4.867639       9.068889   
11      14.212962   16.111391      1.568611         4.038056       9.854444   
12      12.706487   15.006235      7.988472         1.181389      57.492083   
2024   103.218695  132.647195    147.573750       262.577500     254.259306   

                                                  z Anzahl                   \
kat   3: Wind < 5.7      9: Rest         2024 0: <0 ct/kWh 1: PV > 300 W/m2   
Monat                                                                         
1         12.010139   232.509861   294.168194         16.0             19.0   
2         10.222917   250.802222   330.898889          4.0             52.0   
3         12.994861   143.059722   223.840417         12.0            100.0   
4         13.495556   149.649167   277.670278         50.0            115.0   
5         12.456806    81.950556   161.417083         78.0            116.0   
6         12.244028    30.484306   116.154722         66.0            176.0   
7         10.004444    15.949722    67.575000         81.0            183.0   
8          9.239861    13.042639    52.546250         68.0            194.0   
9         10.944306    66.775972   124.374861         40.0            104.0   
10        12.910556    38.615417    73.004861         25.0             44.0   
11        10.030000   137.113472   162.604583         11.0             10.0   
12        10.045278   142.527361   219.234583          8.0              2.0   
2024     136.598750  1302.480417  2103.489722        459.0           1115.0   

                                                  
kat   2: <3,5 ct/kWh 3: Wind < 5.7 9: Rest  2024  
Monat                                             
1               32.0         359.0   318.0   744  
2               47.0         226.0   367.0   696  
3               51.0         368.0   213.0   744  
4               67.0         248.0   240.0   720 

'5.7'